In [66]:
from quant_pl.performance_pl import PerformancePL

import polars as pl
import pandas as pd
from quant_utils.performance import Performance
from quant_utils.db_conn import DB_CONN_JJTG_DATA

In [67]:
query_sql = """
SELECT
  TRADE_DT, PORTFOLIO_NAME, 
  1+ portfolio_ret_accumulated/100 as PORTFOLIO_NAV
FROM
  portfolio_derivatives_ret
WHERE
  `PORTFOLIO_NAME` IN ('知己优选-现金宝', '知己优选-货币小福星') 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)


exec_query耗时:0.0110 s


In [68]:
df = df[["TRADE_DT", "PORTFOLIO_NAME", "PORTFOLIO_NAV"]]

In [69]:
file_path = "f:/931059.CSI.xlsx"
df2 = pl.read_excel(file_path)
df2 = df2.unpivot(index="日期", value_name="PORTFOLIO_NAV", variable_name="PORTFOLIO_NAME")

In [70]:
df2.columns = ["TRADE_DT", "PORTFOLIO_NAME", "PORTFOLIO_NAV"]
df = pl.concat([pl.from_pandas(df), df2])

In [71]:
df = df.to_pandas()
df = df.pivot_table(index="TRADE_DT", columns="PORTFOLIO_NAME", values="PORTFOLIO_NAV")

In [72]:
df.dropna(inplace=True)


In [73]:
result_list = []
for col in df.columns:
    temp = df[col]
    perf = Performance(temp).stats().T
    perf["name"] = col
    result_list.append(perf)
result = pd.concat(result_list)


In [75]:
result.to_excel("f:/jieguo.xlsx", index=False)

In [77]:
df = df/df.iloc[0]

In [80]:
df.to_excel("f:/jieguo2.xlsx")